In [81]:
%matplotlib inline
import os
import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import re

import torch
import numpy as np

from IPython.display import display, Image

import pandas as pd

In [2]:
from output_utils import *

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
pathpattern = 'Laborieux-Arch/results/*/*/*/*/'
runs = getruns(pathpattern=pathpattern)

In [79]:
runsdf = pd.DataFrame(runs)
runsdf[['sparse' in m.lower() for m in runsdf['model']]].sort_values('dir', ascending=False)[['model', 'train', 'test', 'sparse_layers', 'lat_layers', 'lambdas', 'loss', 'T2', 'load_path_convert', 'seed', 'dir']]

,model,train,test,sparse_layers,lat_layers,lambdas,loss,T2,load_path_convert,seed,dir
82,SparseCNN,10.00,10.00,"[-2, -1]",[],"[0.0, 1e-05]",mse,120,,999.00,./Laborieux-Arch/results/EP/mse/2023-08-12/14-...
81,SparseCNN,10.00,10.00,"[-2, -1]",[],"[0.0, 1e-05]",mse,120,,888.00,./Laborieux-Arch/results/EP/mse/2023-08-12/13-...
80,SparseCNN,10.00,10.00,"[-2, -1]",[],"[0.0, 1e-05]",mse,120,,777.00,./Laborieux-Arch/results/EP/mse/2023-08-12/13-...
79,SparseCNN,10.00,10.00,"[-2, -1]",[],"[0.0, 1e-05]",mse,25,,222.00,./Laborieux-Arch/results/EP/mse/2023-08-12/13-...
78,SparseCNN,10.00,10.00,"[-2, -1]",[],"[0.0, 1e-05]",mse,25,,111.00,./Laborieux-Arch/results/EP/mse/2023-08-12/13-...
71,SparseCNN,76.72,76.94,"[-2, -1]",[],"[0.0, 0.1]",mse,25,results/EP/mse/2023-08-03/14-38-36_gpu0,"8,453.00",./Laborieux-Arch/results/EP/mse/2023-08-12/08-...
70,SparseCNN,9.90,10.00,"[-2, -1]",[],"[0.0, 1e-05]",mse,25,,"23,491.00",./Laborieux-Arch/results/EP/mse/2023-08-12/08-...
68,SparseCNN,15.41,10.00,"[-2, -1]",[],"[0.0, 0.1]",mse,25,results/EP/mse/2023-08-03/14-38-36_gpu0,"8,453.00",./Laborieux-Arch/results/EP/mse/2023-08-11/16-...
41,SparseCNN,10.04,10.00,[],[],[],mse,25,,"8,453.00",./Laborieux-Arch/results/EP/mse/2023-08-01/11-...
40,SparseCNN,11.48,10.13,[],[],[],mse,25,,"8,453.00",./Laborieux-Arch/results/EP/mse/2023-08-01/10-...


In [48]:
runsdf.columns

Index(['model', 'task', 'pools', 'archi', 'channels', 'kernels', 'strides',
       'paddings', 'fc', 'act', 'optim', 'lrs', 'wds', 'mmt', 'loss', 'alg',
       'mbs', 'T1', 'T2', 'betas', 'epochs', 'check_thm', 'random_sign',
       'data_aug', 'lr_decay', 'scale', 'save', 'todo', 'load_path', 'seed',
       'device', 'thirdphase', 'softmax', 'same_update', 'cep_debug',
       'train_lateral', 'lat_layers', 'lat_kernels', 'sparse_layers',
       'lat_constraints', 'comp_syn_constraints', 'competitiontype',
       'inhibitstrength', 'lat_init_zeros', 'lat_lrs', 'head_lrs', 'lat_wds',
       'save_nrn', 'load_path_convert', 'convert_place_layers', 'tensorboard',
       'lambdas', 'eps', 'mbs_test', 'nbatches', 'figs', 'jit', 'cpu', 'train',
       'test', 'dir', 'call'],
      dtype='object')

In [25]:
runsdf.iloc[71].dir

'./Laborieux-Arch/results/EP/mse/2023-08-12/08-14-53_gpu0/'

In [82]:
import hopfield_models, lateral_models

In [83]:
lateral_models

<module 'lateral_models' from '/vast/home/rstrauss/repos/Laborieux-Equilibrium-Propagation/lateral_models.py'>

In [84]:
hopfield_models

<module 'hopfield_models' from '/vast/home/rstrauss/repos/Laborieux-Equilibrium-Propagation/hopfield_models.py'>

In [85]:
lateral_models.sparseCNN

hopfield_models.sparseCNN

In [86]:
lateral_models.sparseCNN = hopfield_models.sparseCNN

In [87]:
lateral_models.sparseCNN

hopfield_models.sparseCNN

In [88]:
model = torch.load('./Laborieux-Arch/results/EP/mse/2023-08-12/08-14-53_gpu0/model.pt', map_location=torch.device('cpu'))

In [89]:
model.fc_comp_layers[0].weight

Parameter containing:
tensor([[ 0.0000, -0.0629,  0.0514,  0.0333, -0.0789, -0.0082,  0.0102,  0.0487,
         -0.0337,  0.0018],
        [-0.0629,  0.0000,  0.0314, -0.0589,  0.0131, -0.0356, -0.0445, -0.0488,
         -0.0179, -0.0102],
        [ 0.0514,  0.0314,  0.0000,  0.0623, -0.0500, -0.0510,  0.1165, -0.0576,
         -0.0182,  0.0487],
        [ 0.0333, -0.0589,  0.0623,  0.0000,  0.1089,  0.0475, -0.0346,  0.0655,
          0.0074,  0.0228],
        [-0.0789,  0.0131, -0.0500,  0.1089,  0.0000, -0.0287, -0.0224,  0.0575,
          0.0632, -0.0481],
        [-0.0082, -0.0356, -0.0510,  0.0475, -0.0287,  0.0000,  0.0500, -0.0544,
          0.0199,  0.0093],
        [ 0.0102, -0.0445,  0.1165, -0.0346, -0.0224,  0.0500,  0.0000,  0.0467,
          0.0193,  0.0267],
        [ 0.0487, -0.0488, -0.0576,  0.0655,  0.0575, -0.0544,  0.0467,  0.0000,
          0.0167, -0.0003],
        [-0.0337, -0.0179, -0.0182,  0.0074,  0.0632,  0.0199,  0.0193,  0.0167,
          0.0000, -0.0054

In [90]:
feat = model.conv_comp_layers[0].weight
print(feat[:,:,2,2])
print(feat[:,:,0,0])

tensor([[ 0.0000, -0.0079, -0.0097,  ...,  0.0037,  0.0007, -0.0063],
        [-0.0079,  0.0000, -0.0121,  ...,  0.0079, -0.0175,  0.0065],
        [-0.0097, -0.0121,  0.0000,  ...,  0.0178,  0.0105, -0.0041],
        ...,
        [ 0.0037,  0.0079,  0.0178,  ...,  0.0000, -0.0121, -0.0078],
        [ 0.0007, -0.0175,  0.0105,  ..., -0.0121,  0.0000,  0.0204],
        [-0.0063,  0.0065, -0.0041,  ..., -0.0078,  0.0204,  0.0000]],
       grad_fn=<SelectBackward0>)
tensor([[-1.0000e+00, -7.8927e-03, -9.7482e-03,  ...,  3.6985e-03,
          6.7520e-04, -6.3258e-03],
        [-7.8927e-03, -1.0000e+00, -1.2144e-02,  ...,  7.9263e-03,
         -1.7559e-02,  6.5416e-03],
        [-9.7482e-03, -1.2144e-02, -1.0000e+00,  ...,  1.7814e-02,
          1.0495e-02, -4.1399e-03],
        ...,
        [ 3.6985e-03,  7.9263e-03,  1.7814e-02,  ..., -1.0000e+00,
         -1.2118e-02, -7.8519e-03],
        [ 6.7520e-04, -1.7559e-02,  1.0495e-02,  ..., -1.2118e-02,
         -1.0000e+00,  2.0338e-02],
    

In [91]:
model

sparseCNN(
  (synapses): ModuleList(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (lat_syn): ModuleList()
  (conv_comp_layers): ModuleList(
    (0): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (fc_comp_layers): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [92]:
model.conv_comp_layers

ModuleList(
  (0): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)

In [76]:
for param in model.parameters():
    print(param.sum())

tensor(-1.3139, grad_fn=<SumBackward0>)
tensor(4.3100, grad_fn=<SumBackward0>)
tensor(-335.8656, grad_fn=<SumBackward0>)
tensor(1.7050, grad_fn=<SumBackward0>)
tensor(-1002.0408, grad_fn=<SumBackward0>)
tensor(5.4406, grad_fn=<SumBackward0>)
tensor(-524.4692, grad_fn=<SumBackward0>)
tensor(10.3719, grad_fn=<SumBackward0>)
tensor(-45.4680, grad_fn=<SumBackward0>)
tensor(2.5025, grad_fn=<SumBackward0>)
tensor(-11951.8516, grad_fn=<SumBackward0>)
tensor(0.0575, grad_fn=<SumBackward0>)
tensor(0.4172, grad_fn=<SumBackward0>)
tensor(-1.8029e-05, grad_fn=<SumBackward0>)


In [78]:
for s in model.synapses:
    print(s.weight.sum())
    print(s.bias.sum())

tensor(-1.3139, grad_fn=<SumBackward0>)
tensor(4.3100, grad_fn=<SumBackward0>)
tensor(-335.8656, grad_fn=<SumBackward0>)
tensor(1.7050, grad_fn=<SumBackward0>)
tensor(-1002.0408, grad_fn=<SumBackward0>)
tensor(5.4406, grad_fn=<SumBackward0>)
tensor(-524.4692, grad_fn=<SumBackward0>)
tensor(10.3719, grad_fn=<SumBackward0>)
tensor(-45.4680, grad_fn=<SumBackward0>)
tensor(2.5025, grad_fn=<SumBackward0>)


In [80]:
print(model.conv_comp_layers[0].weight.sum())

tensor(-11951.8516, grad_fn=<SumBackward0>)
